# This is a basic tutorial to run Kalkayotl
To be able to use Kalkayotl you will need to follow the installation steps provided in the GitHub [page](https://github.com/olivares-j/Kalkayotl), and activate its environment.

You can either follow the step-by-step instructions of this tutorial (only for the King prior family) or edit and run the `example.py`.

First, we load the libraries:

In [1]:
from __future__ import absolute_import, unicode_literals, print_function
import sys
import os
import numpy as np
from kalkayotl import Inference

If there are any errors then review the installation steps of Kalkayotl and/or Jupyterlab.

Next, we define the data file and directory, we create the latter if it does not exists.

In [2]:
dir_out    = os.getcwd() + "/Example/"
os.makedirs(dir_out,exist_ok=True)

We provide the data file, which must be in CSV format. The names of the columns must be those of the standard Gaia DR2. The only exception is the source ID, by default it is set to "source_id" as in Gaia data, but you will be able to provide another name. More of it when we load the data.

In [3]:
file_data = dir_out + "Ruprecht_147.csv"

## Knobs
Now we define some of the code parameters.

### Sampler parameters 

_Chains_: it refers to the number of HMC chains that the sampler will create. To analyze convergence we need at least two. More chains will provide more samples but will also consume more resources.

In [4]:
chains = 2

_Cores_: it refers to the number of computers cores or processors that will be used to run the HMC sampler. The best performance will vary in different machines. The best option is to use one core per chain.

In [5]:
cores  = 2

_Burning_iterations_: it refers to the number of iterations that the HMC sampler will use to adapt its parameters. The rule here is that more iterations will help to improve the performance of the sampler. This is a parameter that you may need to increase if convergence problems arise. These iterations will be discarded to avoid biasing the parameters estimates.

In [6]:
burning_iters   = 10000

_Sample_iterations_: it refers to the number of actual samples that will be delivered by the HMC sampler. As explained in the paper, the number of samples depend on the precision that you need for the model parameters. More samples improve the precision but also take more time. Adapt this value according to your needs.

In [7]:
sample_iters    = 2000

The next two parameters refer to the initialization of the sampler positions. These are the initial positions of the HMC chains. Although theoretically the sampler must converge in spite of the values of the initial positions, in practice if these are far away from the true parameter values, then the likelihood is not able improve by small movements of the parameters positions, and the sampler is basically lost and takes a lot of time to converge. Since we do not want to waste our time, we provide the sampler with a roughly good starting point.

After testing several of the initialization modes provided by PyMC3, I found that the best one is the 'advi+adapt_diag' with 500000 iterations. This scheme performs variational inference to find the best parameters positions. Although it is the best of the initialization schemes, at least for the particular case of Kalkyotl, it is still prone to failures. In some cases if one or several of the distances to the stars fail to fall within the "field-of-view" of its parallax uncertainties (i.e. fall beyond 5-sigma) then the initialization will fail with errors like: "Bad initial energy" or ".rvel is zero". In this cases launch the code again. Hopefully the initial position of the new run will fall closer to the "field-of-view" of the uncertainties. This type of problem is more recurrent in models with high number of parameters (i.e. stars).

In [8]:
init_mode = 'advi+adapt_diag'
init_iter = 500000 

_Target_acceptance_: it refers to an internal parameter of the HMC. It is recommended to be larger than 0.7. Smaller values work better for simpler problems, while larger values help to improve the convergence of the sampler but also increases the computing time. Usually the more complex prior families require a larger value (i.e. closer to 1). For this reason this parameter is defined for each prior family. But you are free to increase its value in case of convergence problems.

For the King prior we set it to:

In [9]:
target_accept = 0.95

### Output statistics

_Statistics_: It refers to the type of statistics that will be computed from the posterior samples. Options are "mean","median" and "mode". The quantiles refer to the lower and upper values of the uncertainties. One sigma uncertainties will correspond to [0.16,0.84], while two sigma uncertainties will be [0.025,0.975]

In [10]:
statistic = "mean"
quantiles = [0.025,0.975]

### Model configuration

_Transformation_: it refers to the space in which Kalkayotl will work, either in the distance space (choose "pc") or the parallax space (choose "mas"). These units will be the same in which the hyper-parameters must be specified.

In [11]:
transformation = "pc"

_Zero_point_: it refers to the parallax zero point of the Gaia data. You can provide either a scalar or a vector of the same dimension as the valid sources in your data set. We use the Lindegren et al. 2018 value.

In [12]:
zero_point = -0.029 

_Parametrization_: it refers to the type of parametrization of the Hierarchical model. It is known that this type of models face problems when its parameters are inferred using HMC samplers. To improve performance two types of parameterizations are provided: "central" and "non-central". While the first one works better when the data set is highly informative (nearby clusters stars with narrow parallax uncertainties), the last one works better for low informative data sets, like those of the farthest stars and clusters. In the case of Ruprecht 147 we use the central parameterizations.

In [13]:
parametrization="central"

_Independent_measurements_: In the Gaia astrometric data the measurements of stars are spatially correlated.
This parameter controls if the data is assumed to be independent (i.e. the spatial correlations are neglected) or not (i.e. the spatial correlations are taken into account. Setting indep_measures=False implies that the spatial correlations will be taken into account. The default model for this correlations is the one provided by Vasiliev et al. 2019.

In [14]:
indep_measures = False

### Prior configuration

Here we will only show how to configure the King prior family. The rest of the families are configured in a similar way since most of the parameters are shared. 

_Type_: We use a valid prior family name: "Uniform", "Gaussian", "King", "EFF", or "GMM".

_Parameters_: It must be a dictionary with the names of the parameters (see the `example.py` file for the names of other prior parameters). You can either use a number, in which case the parameter will be fixed to that value throughout the inference, or set it to None, in this case the value will be inferred as well.

_Hyper_alpha_: It refers to the hyper-parameter of the location prior. You must provide a list with the location and scale of the Gaussian distribution that will be used as prior for the location parameter (i.e. the distance).

_Hyper_beta_: It corresponds to the hyper-parameter of the scale prior. It correspond to the typical size of the cluster. Here we use a rather large value. But if you have more constraining information use it if you face convergence problems.

_Hyper_gamma_: It corresponds to the hyper-parameter of the prior for the tidal radius. Is similar to the hyper_beta but it is now expressed in units of core radius and restricted to be larger than one. 

_Hyper_delta_: It is only used in the GMM prior family so we set it to None.

_Burning_iters_: This is the number of iterations. We keep it within the prior for compatibility to the `example.py` file, where all prior families can be run at once.

_Target_accept_: Similarly to the previous one this parameter is kept within the prior dictionary for compatibility. It is explained above in the sampler parameters.

We set the prior as a dictionary for simplicity, but this is not required by the code.

In [15]:
prior = {
 "type":"King",         
 "parameters":{"location":None,"scale":None,"rt":None},
 "hyper_alpha":[305.,30.], 
 "hyper_beta":[50.], 
 "hyper_gamma":[50.],
 "hyper_delta":None,
 "burning_iters":burning_iters,
 "target_accept":target_accept}

## Running Kalkayotl

First we create the output directory specific for this prior. Again this is for compatibility with the `example.py` file.

In [16]:
dir_prior = dir_out + prior["type"] + "/"
os.makedirs(dir_prior,exist_ok=True)

We initialize the inference module with the model, sampler and prior parameters.

For now, Kalkayotl only works for computing distances, so we set dimension to 1.

In [17]:
p1d = Inference(dimension=1,
                prior=prior["type"],
                parameters=prior["parameters"],
                hyper_alpha=prior["hyper_alpha"],
                hyper_beta=prior["hyper_beta"],
                hyper_gamma=prior["hyper_gamma"],
                hyper_delta=prior["hyper_delta"],
                dir_out=dir_prior,
                transformation=transformation,
                zero_point=zero_point,
                indep_measures=indep_measures,
                parametrization=parametrization)

Now we load the data. Other options for the load_data function beside that of the data file include the `id_name` keyword to use in case you have different ID names as from those in Gaia data. If so you must provide a string containing the column name of the IDs. By default it uses 'source_id'. Finally, the `corr_func` keyword refers to the type of correlation function to use for the spatial correlations. The options are 'Vasiliev+2019', the default one, and 'Lindegren+2018'.

In [18]:
p1d.load_data(file_data)

Using Vasiliev+2019 spatial correlation function
Data correctly loaded


Now we configure the model

In [19]:
p1d.setup()

Configuring King prior
Using central parametrization.


We are now ready to run the sampler with all the previous parameters:

In [20]:
p1d.run(sample_iters=sample_iters,
		burning_iters=prior["burning_iters"],
		init=init_mode,
		n_init=init_iter,
		target_accept=prior["target_accept"],
		chains=chains,
		cores=cores)

Computing posterior


Auto-assigning NUTS sampler...
Initializing NUTS using advi+adapt_diag...
Average Loss = 5.2947e+18:   4%|▍         | 21699/500000 [00:21<07:58, 1000.41it/s]
Convergence achieved at 21700
Interrupted at 21,699 [4%]: Average Loss = 3.2882e+19
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [1D_source, 1D_x, 1D_scl, 1D_loc]
Sampling 2 chains: 100%|██████████| 24000/24000 [04:48<00:00, 83.30draws/s] 
There were 70 divergences after tuning. Increase `target_accept` or reparameterize.
There were 45 divergences after tuning. Increase `target_accept` or reparameterize.
There were 70 divergences after tuning. Increase `target_accept` or reparameterize.
There were 45 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 10% for some parameters.


In some cases the sampler does not perform well and there are few effective samples and or some divergences. To improve the performance we can increase the number of burning iterations, and/or reparameterize (i.e. provide more constraining priors). A few divergences (<100) in the King prior are not generally a problem. If you want to fully remove them you will have to increase the burning iterations.

After running the sampler we load the chains. This function comes at hand when you have already run the model
but you want to reanalyze or makes some plots without running again the model. In this latter case simply comment the previous `p1d.run()` function.

In [21]:
p1d.load_trace(sample_iters=sample_iters)

Loading existing chains ... 


Once the chains/traces are loaded we analyze their convergence

In [22]:
p1d.convergence()

Computing convergence statistics ...
Gelman-Rubin statistics:
1D_loc : 1.0000
1D_source : 0.9999
1D_scl : 0.9999
1D_x : 1.0011
1D_rt : 1.0011
Effective sample size:
1D_loc : 334.9014
1D_source : 2297.1414
1D_scl : 726.5903
1D_x : 970.5935
1D_rt : 970.5935


The Gelman-Rubin statistic must be near one, which is the case. Let's take a look at the plots.

## Plots

Once we are satisfied with the convergence of the sampler we can make further checks by analyzing the trace plots.

We start by plotting the chains. The `plot_chains` function takes as optional argument the `IDs` keyword. If not supplied the function will create only plots with the traces of the cluster parameters. If you pass valid IDs as a list of strings it will also create the trace plots for individual sources for which the IDs were provided. The output plots will be in the `dir_out` directory under the name "Traces.pdf".

In [23]:
p1d.plot_chains(IDs=['4087735025198194176'])

Plotting traces ...


In [24]:
from IPython.display import IFrame
file_traces = "Example/King/Traces.pdf"
IFrame(file_traces,width=700, height=350)

The plot shows that source '4087735025198194176' has a nice mixing of the two chains. Concerning the cluster parameters we see that the location has still some correlations, which is causing the low effective sample size. Both the scale and rt parameters are fine, with some correlation still. The chains look fine and the Gelman-Rubin is also correct, if you still want to further improve convergence you can try with larger chains and more constraining priors.

## Samples and statistics

Kalkayotl can also provide the statistics and quantiles of all parameters in the model. The function `save_statistics` will create two output CSV files, within the `dir_out` directory. These files will be named Sources_{statistic}.csv and Cluster_{statistic}. csv and will contain the statistics of the individual sources and the cluster parameters, respectively.

In [25]:
p1d.save_statistics(statistic=statistic,quantiles=quantiles)

Saving statistics ...


We also save the samples from the posterior into an HDF5 file. This latter is more compressed than the CSV files. The file will be created in the `dir_out` directory specified in the initialization.

In [26]:
p1d.save_samples()

Saving samples ...


## Evidence

As explained in the paper, the evidence module of Kalkayotl is there only to give you some help when deciding which prior family is the best one for your particular data set. It is computationally expensive, so if you have a large data set run the evidence only in a subsample of your data set (the `N_samples` parameter). The `M_samples` parameter controls how many samples fill be taken from the prior and then marginalized; a larger value implies more accurate results but also longer computing time. The rest of the parameters of the `evidence` function include specific parameters of the _dynesty_ code, these are the number of live points (`nlive`) and the convergence tolerance (`dlogz`). At the end an output file called 'Evidence.csv' will be created in the `dir_out` directory. 

In [27]:
p1d.evidence(M_samples=1000,dlogz=1.0,nlive=100)

Estimating evidence of prior:  King


/home/javier/anaconda3/envs/kalkayotl/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:998: VisibleDeprecationWarning: The signature of <bound method king_gen._rvs of <kalkayotl.Priors.king_gen object at 0x7f983b68b898>> does not contain a "size" keyword.  Such signatures are deprecated.
  np.VisibleDeprecationWarning)


Log Z: 128.896 +/- 0.342


## Extract samples

Finally, here is a piece of code showing how to extract the samples from the Samples.h5 file. In addition it will print the mean and standard deviation of the samples obtained for each source.

In [28]:
import h5py
file_distances = dir_out + "/King/Samples.h5"
hf = h5py.File(file_distances,'r')
srcs = hf.get("Sources")

n_samples = 100
samples = np.empty((len(srcs.keys()),n_samples))
#-------- loop over array and fill it with samples -------
for i,ID in enumerate(srcs.keys()):
	#--- Extracts a random choice of the samples --------------
	samples[i] = np.random.choice(np.array(srcs.get(str(ID))),
							size=n_samples,replace=False)
	#----------------------------------------------------------

	print("Source {0} at {1:3.1f} +/- {2:3.1f} pc.".format(ID,
										samples[i].mean(),
										samples[i].std()))

#- Close HDF5 file ---
hf.close()

Source 4081223953548454656 at 284.5 +/- 7.8 pc.
Source 4081935822925977088 at 304.3 +/- 7.1 pc.
Source 4082132154460682752 at 294.2 +/- 5.1 pc.
Source 4082207638503600256 at 302.2 +/- 7.3 pc.
Source 4082219904923606016 at 300.8 +/- 4.4 pc.
Source 4082432458578047232 at 306.4 +/- 7.1 pc.
Source 4082446851004435072 at 302.8 +/- 6.9 pc.
Source 4083221839189590528 at 306.5 +/- 10.1 pc.
Source 4083614807222135936 at 291.4 +/- 6.7 pc.
Source 4083991252520671232 at 320.1 +/- 5.2 pc.
Source 4084099039017310848 at 305.9 +/- 6.1 pc.
Source 4084119143773162496 at 300.7 +/- 7.1 pc.
Source 4084126118799904640 at 304.4 +/- 6.1 pc.
Source 4084174462953441792 at 300.2 +/- 4.2 pc.
Source 4084194769558646528 at 300.8 +/- 7.4 pc.
Source 4084473255232485248 at 281.5 +/- 4.8 pc.
Source 4084555168851639680 at 300.2 +/- 4.3 pc.
Source 4084585817738646400 at 298.0 +/- 5.2 pc.
Source 4084645500601105536 at 301.7 +/- 4.6 pc.
Source 4084751706554608000 at 307.7 +/- 3.6 pc.
Source 4084757199808535296 at 304.9 +/-